<a href="https://colab.research.google.com/github/SohaliBaisla/Hindi-Fake-News-Fact-Checker/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
size=[0.1,0.2,0.3,0.4]
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Embedding

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install inltk
from inltk.inltk import setup
#setup('hi')
from inltk.inltk import tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Dataset/acm_datasetFinalUsed.csv")
df.columns =['heading', 'text', 'link','output']
df=df.drop(['heading','link'], axis = 1)
df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df

,text,output
0,हैशटैग तस्वीर शेयर ओवैसी पोस्टर कालिख लगी देखी...,1
1,हैशटैग तस्वीर शेयर ओवैसी पोस्टर कालिख लगी देखी...,1
2,हैशटैग तस्वीर शेयर ओवैसी पोस्टर कालिख लगी देखी...,1
3,तस्वीर रिवर्स सर्च हमें 'जागरण' 24 सितंबर 2019...,0
4,मामले और=_खबरें प्रकाशित थीं.,0
...,...,...
2201,हालांकि पक्षियों टॉवरों से==_टकराकर मौत खासकर ...,0
2202,"दावा भारत पक्षियों मौत ट्रायल है, बेबुनियाद भी...",0
2203,सत्ता को आईना दिखाने वाली पत्रकारिता का कॉरपोर...,0
2204,तस्वीर==_फे़सबुक शेयर,1


In [ ]:
from google.colab import files
uploaded = files.upload()
file1 = open("final_stopwords.txt","r+")
Lines = file1.readlines()
stopword=[]
for line in Lines:
  stopword.append(line)

from google.colab import files
uploaded = files.upload()
file1 = open("final_stopwords2.txt","r+")
Lines = file1.readlines()
stopword3=[]
for line in Lines:
  stopword.append(line)
stopword2 = ['','.','!','?', 'अर्थात', 'कुछ', 'तेरी', 'साबुत', 'अपनि', 'हूं', 'काफि', 'यिह', 'जा' ,'दे', 'देकर' ,'रह', 'कह' , 'कर' , 'कहा', 'बात' , 'जिन्हों', 'किर', 'कोई', 'हे', 'कोन', 'रहा', 'सब', 'सो', 'तक', 'इंहें', 'इसकि', 'अपनी', 'दबारा', 'सभि', 'होते', 'भीतर', 'निचे', 'घर', 'उन्हें', 'उन्ह' , 'मेरे' , 'था', 'व', 'इसमें', 'उसी', 'बिलकुल', 'होति', 'गया', 'सकता', 'अपना', 'लिये', 'उसका', 'पर', 'दवारा', 'गए', 'है', 'कितना', 'भि', 'लिए', 'वुह ', 'ना', 'किसि', 'परन्तु', 'किन्हें', 'बहुत', 'भी', 'तुम्हारे', 'निहायत', 'उन्हीं', 'वहिं', 'हैं', 'उन्हों', 'इतयादि','यहाँ', 'तब', 'पूरा', 'क्योंकि', 'कौनसा', 'आप', 'हुअ', 'ऐसे', 'एस', 'कारण', 'अप', 'पहले', 'तुम', 'जेसा', 'तिस', 'लेकिन', 'कहते', 'मगर', 'करता', 'संग', 'सभी', 'जीधर', 'किंहों', 'हि', 'द्वारा', 'हुआ', 'तू', 'जिंहें', 'उसने', 'पास', 'वहां', 'वह', 'किंहें', 'इंहों', 'मुझ', 'कुल', 'तिंहों', 'का', 'मेरी', 'तेरे', 'उनके', 'क्या', 'जहाँ', 'काफ़ी', 'वर्ग', 'वरग','बही', 'ये', 'जिस', 'इसि', 'हुई', 'साम्हने', 'नहिं', 'जैसे', 'वहीं', 'दिया', 'अभी', 'यहि', 'वग़ैरह', 'उनकि', 'न', 'जा','बनि', 'हें', 'यिह ', 'उन', 'को', 'तिन्हों', 'उन्होंने', 'तुझे', 'उसे', 'होने', 'इन्हीं', 'थे', 'उंहिं', 'अपने', 'में', 'फिर','यही', 'नीचे', 'होती', 'तिसे', 'हम', 'यदि', 'सारा', 'कर', 'सकते', 'कोइ', 'और', 'जिंहों', 'तिंहें', 'दूसरे', 'जब', 'रहे','अत', 'मानो', 'जिन', 'बाद', 'उनका', 'किया', 'या', 'उनकी', 'कौन', 'ऐसा', 'सबसे', 'अनुसार', 'दुसरे', 'इन', 'अदि','जिसे', 'उसकी', 'इत्यादि', 'करना', 'यहां', 'हुए', 'तेरा', 'आदि', 'पर  ', 'वाले', 'कहता', 'किन्हों', 'किसे', 'जिन्हें', 'मे','होता', 'करने', 'साभ', 'अभि', 'उसको', 'कई', 'बनी', 'के', 'इन्हें', 'वहाँ', 'कोनसा', 'कइ', 'इनका', 'थि', 'बाला','ऱ्वासा', 'हो', 'उंहें', 'दुसरा', 'वे', 'भितर', 'जेसे', 'एवं', 'अंदर', 'दो', 'साथ', 'करें', 'जिधर', 'तरह', 'उसि', 'इस', 'एसे', 'तिन', 'नहीं', 'से','न','उनको', 'किस', 'किसी', 'इसी', 'मैं', 'यह', 'हुइ', 'ले', 'कि', 'की', 'इसलिये', 'रवासा', 'ने', 'जैसा', 'वह ', 'तिन्हें', 'वुह', 'उस', 'उंहों', 'वगेरह', 'उसके', 'मुझे', 'करते', 'जितना', 'जहां', 'इन्हों', 'इसके', 'होना', 'इसका', 'इंहिं', 'एक', 'जो', 'पे', 'ही', 'तो', 'थी', 'रखें', 'इसे', 'इन ', 'के', 'बहि', 'पुरा', 'ओर', 'इसकी']
stopword=stopword+stopword2+stopword3

Saving final_stopwords.txt to final_stopwords.txt


Saving final_stopwords2.txt to final_stopwords2.txt


In [ ]:

for m in size:
      x_train, x_test = train_test_split(df, test_size=m)

      x_tr=[]
      for i in x_train['text'].astype(str):
        if i not in stopword:
          x_tr.append(i)
      x_ts=[]
      for i in x_test['text'].astype(str):
        if i not in stopword:
          x_ts.append(i)


      y_train=x_train['output']

      y_test=x_test['output']
      
      from sklearn.feature_extraction.text import TfidfVectorizer
      vectorizer=TfidfVectorizer()

      x_train=vectorizer.fit_transform(x_train['text'].astype(str))
 
      x_test=vectorizer.transform(x_test['text'].astype(str))

      
      x_test = x_test.toarray()
      x_train = x_train.toarray()
      
      #DL-----------------------------------------------------
      
      MAX_NB_WORDS = 50000
      EMBEDDING_DIM = 100
      model = Sequential()
      from keras.callbacks import ModelCheckpoint, EarlyStopping
      from keras.layers import Embedding
      model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x_train.shape[1]))
      model.add(Dropout(0.2))
      model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
      model.add(Dense(1, activation='sigmoid'))
      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

      epochs = 100
      batch_size = 64

      history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,callbacks=[EarlyStopping(monitor='accuracy', patience=3, min_delta=0.0001)])
      accr = model.evaluate(x_test,y_test)
      #accuracy.append(accr[1])
      print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Epoch 1/100
32/32 [==============================] - 205s 6s/step - loss: 0.0000e+00 - accuracy: 0.5622
Epoch 2/100
32/32 [==============================] - 201s 6s/step - loss: 0.0000e+00 - accuracy: 0.5698
Epoch 3/100
32/32 [==============================] - 200s 6s/step - loss: 0.0000e+00 - accuracy: 0.5698
Epoch 4/100
32/32 [==============================] - 201s 6s/step - loss: 0.0000e+00 - accuracy: 0.5698
Epoch 5/100
7/7 [==============================] - 4s 529ms/step - loss: 0.0000e+00 - accuracy: 0.6471
Test set
  Loss: 0.000
  Accuracy: 0.647
Epoch 1/100
28/28 [==============================] - 175s 6s/step - loss: 0.0000e+00 - accuracy: 0.5794
Epoch 2/100
28/28 [==============================] - 179s 6s/step - loss: 0.0000e+00 - accuracy: 0.5788
Epoch 3/100
28/28 [==============================] - 174s 6s/step - loss: 0.0000e+00 - accuracy: 0.5788
Epoch 4/100
14/14 [==============================] - 8s 521ms/step - loss: 0.0000e+00 - accuracy: 0.5724
Test set
  Loss: 0.000
